In [1]:
import os
import argparse
import time

import matplotlib as mpl
mpl.use('TkAgg') # sets the backend for matplotlib
import matplotlib.pyplot as plt

from utils.plot import redraw_obstacles, set_non_blocking
from commonroad.common.file_reader import CommonRoadFileReader
from commonroad.visualization.draw_dispatch_cr import draw_object

import numpy as np


In [2]:
def plotScenario(filename, nrun):
    scenario, _ = CommonRoadFileReader(filename).open()

    set_non_blocking()  # ensures interactive plotting is activated
    plt.style.use('classic')
    inch_in_cm = 2.54
    figsize = [30, 8]

    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    fig.gca().axis('equal')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib

    # inital plot including the lanelet network
    draw_object(scenario, handles=handles, draw_params={'time_begin': 2000})
    fig.canvas.draw()
    plt.gca().autoscale()
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    nrun= 2500
    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(0, nrun):
        # ...
        # change positions of obstacles
        # ...
        redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=None, draw_params={'time_begin':i})
        print('fps:', nrun/(time.time()-t1))

In [3]:
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/USA_Lanker-1_1_T-1.xml'
file_path = '/home/rong/VAE-Motion-Planning/scenarios/commonroad_data/highway.cr.xml'
run = 100
plotScenario(file_path, run)

fps: 13214.168695590808
fps: 7345.777867447918
fps: 5084.685271823041
fps: 3667.431927463909
fps: 2883.2007505402344
fps: 2427.8302489659604
fps: 2088.8176177053556
fps: 1919.3608378459908
fps: 1746.200312045111
fps: 1595.7080770455807
fps: 1426.9941597505222
fps: 1307.4955525305156
fps: 1217.8201148163762
fps: 1150.3748567895339
fps: 1087.7457235033557
fps: 1037.9137199378226
fps: 986.7439593014792
fps: 931.4018309224491
fps: 879.2290177371702
fps: 843.6458920077969
fps: 803.3114410761095
fps: 773.7570661553821
fps: 745.2710877240964
fps: 721.0260505674582
fps: 700.8618806628009
fps: 681.8751853723955
fps: 664.7976755514013
fps: 649.4348808644334
fps: 635.767281536698
fps: 614.3351228513817
fps: 595.0933357668815
fps: 579.6731621104212
fps: 568.9730680097907
fps: 557.4362237642532
fps: 546.9155650172364
fps: 536.9631768791862
fps: 529.1695627579641
fps: 520.6393076679324
fps: 512.5345825790268
fps: 505.3619669257457
fps: 497.90134480599755
fps: 488.19978378297844
fps: 477.022206924650

TclError: invalid command name "pyimage10"

In [4]:
scenario, _ = CommonRoadFileReader(file_path).open()

In [5]:
def create_draw_params() -> dict:
    shape_parameters = {'opacity': 1.0,
                                   'facecolor': '#000000',
                                   'edgecolor': '#000000',
                                   'linewidth': 0.5,
                                   'zorder': 20}

    draw_params =  { 'time_begin': 200,
#          'time_end': 10,
         'antialiased': True,
         'scenario':
                {'dynamic_obstacle':
                    {'occupancy':
                        {'draw_occupancies': -1,
                         'shape': shape_parameters
                        },
                     'shape': shape_parameters,
                     'draw_shape': True,
                     'draw_icon': False,
                     'draw_bounding_box': True,
                     'show_label': False,
                     'trajectory_steps': 40,
                     'zorder': 20
                    },
                'static_obstacle':
                    {'shape': shape_parameters},
                'lanelet_network':
                    {'lanelet':
                        {'left_bound_color': '#000000',
                         'right_bound_color': '#000000',
                         'center_bound_color': '#dddddd',
                         'draw_left_bound': True,
                         'draw_right_bound': True,
                         'draw_center_bound': False,
                         'draw_border_vertices': False,
                         'draw_start_and_direction': False,
                         'show_label': False,
                         'draw_linewidth': 0.5,
                         'fill_lanelet': False,
                         'facecolor': '#c7c7c7'
                        }
                    }
                }
         }
    # ensure that parameters are also available on higher levels
    draw_params.update(draw_params['scenario'])
    draw_params['shape'] = shape_parameters
    draw_params['shape'].update(draw_params['scenario']['static_obstacle']['shape'])
    draw_params['occupancy'] = draw_params['scenario']['dynamic_obstacle']['occupancy']
    draw_params['static_obstacle'] = draw_params['scenario']['static_obstacle']
    draw_params['dynamic_obstacle'] = draw_params['scenario']['dynamic_obstacle']
#     draw_params['trajectory'] = draw_params['scenario']['dynamic_obstacle']['trajectory']
    draw_params['lanelet_network'] = draw_params['scenario']['lanelet_network']
    draw_params['lanelet'] = draw_params['scenario']['lanelet_network']['lanelet']
    draw_params['scenario']['lanelet'] = draw_params['scenario']['lanelet_network']['lanelet']

    return draw_params

from commonroad.common.util import Interval
from commonroad.scenario.scenario import Scenario
from typing import List, Dict, Union
def redraw_obstacles(scenario: Scenario, handles: Dict[int,List[mpl.patches.Patch]], figure_handle: mpl.figure.Figure,
                     draw_params=None, plot_limits: Union[List[Union[int,float]], None] = None) -> None:
    """
    This function is used for fast updating dynamic obstacles of an already drawn plot. Saves about 80% time compared to a complete plot.
    Deletes all dynamic obstacles which are specified in handles and draws dynamic obstacles of a scenario.

    :param scenario: scenario from which obstacle should be plotted
    :param handles: dict of obstacle_ids and corresponding patch handles (generated by draw_object function)
    :param plot_limits: axis limits for plot [x_min, x_max, y_min, y_max]
    :param figure_handle: figure handle of current plot
    :return: None
    """
    #remove dynamic obstacle from current plot
    for handles_i in handles.values():
        for handle in handles_i:
            if handle is not None:
                handle.remove()
    handles.clear()
    # redraw dynamic obstacles
    if plot_limits is not None:
        draw_object(scenario.obstacles_by_position_intervals([Interval(plot_limits[0], plot_limits[1]),
                                                              Interval(plot_limits[2], plot_limits[3])], time_step=draw_params['time_begin']),
                    draw_params=draw_params, plot_limits=plot_limits, handles=handles)
    else:
        draw_object(scenario.obstacles, draw_params=draw_params, plot_limits=plot_limits, handles=handles)

    # update plot
    ax = figure_handle.gca()
    for handles_i in handles.values():
        for handle in handles_i:
            if handle is not None:
                ax.draw_artist(handle)

    if mpl.get_backend() =='TkAgg':
        figure_handle.canvas.draw()

    elif mpl.get_backend() =='Qt5Agg':
        figure_handle.canvas.update()
    else:
        try:
            figure_handle.canvas.update()
        except:
            raise Exception('<plot_helper/redraw_dynamic_obstacles> Backend for matplotlib needs to be \'Qt5Agg\' or \'TkAgg\' but is'
                        '\'%s\'' % mpl.get_backend())

    figure_handle.canvas.flush_events()

inch_in_cm = 10

def plotWindow(scenario, duration, draw_parameters, size, pos):
    figsize = size
    fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
    fig.gca().axis('equal')
#     fig.gca().axis('off')
    handles = {}  # collects handles of obstacle patches, plotted by matplotlib
    plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
    draw_object(scenario, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
#     plt.set_cmap('Greys')
#     plt.gray()
    fig.canvas.draw()
    # plt.gca().autoscale()
    plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
    plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
    for handles_i in handles:
        if not handles_i:
            handles.pop()

    t1 = time.time()
    # loop where obstacle positions are updated
    for i in range(duration[0], duration[1]):
        # ...
        # change positions of obstacles
        # ...
        draw_params['time_begin'] = i
        redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
        print('fps:', (i - duration[0])/(time.time()-t1))
draw_params = create_draw_params()

In [6]:
data = plotWindow(scenario,  [300, 301], draw_params, [200, 40], [100, -5])

fps: 0.0


In [9]:
data = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8)
data = data.reshape(fig.canvas.get_width_height()[::-1] + (4,))
data= data[:, : ,1]

In [10]:
# now display the array X as an Axes in a new figure
fig2 = plt.figure()
ax2 = fig2.add_subplot(111, frameon=False)
ax2.imshow(data)
plt.show()

In [11]:
x= data[:, : ,1]

IndexError: too many indices for array

In [389]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111, frameon=False)
ax2.imshow(x)
plt.gray()
plt.show()
ax2.axis('off')

# ax2.spines['top'].set_visible(False)
# ax2.spines['right'].set_visible(False)
# ax2.spines['bottom'].set_visible(False)
# ax2.spines['left'].set_visible(False)

(-0.5, 1599.5, 319.5, -0.5)

In [8]:
size = [200, 40]
pos = [100, -5]
duration = [300, 301]

figsize = size
fig = plt.figure(figsize=(figsize[0] / inch_in_cm, figsize[1] / inch_in_cm))
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)
# fig.gca().axis('equal')
# fig.gca().axis('off')
handles = {}  # collects handles of obstacle patches, plotted by matplotlib
plot_limits = [pos[0] - size[0]/2, pos[0] + size[0]/2, pos[1] +-size[1]/2, pos[1] + size[1]/2]
draw_object(scenario, handles=handles, draw_params=draw_params, plot_limits=plot_limits)
# plt.set_cmap('Greys')
plt.gray()
fig.canvas.draw()
fig.tight_layout(pad=0)
# plt.gca().autoscale()
plt.xlim(pos[0] - size[0]/2, pos[0] + size[0]/2)
plt.ylim( pos[1] -size[1]/2, pos[1] + size[1]/2)
for handles_i in handles:
    if not handles_i:
        handles.pop()

t1 = time.time()
# loop where obstacle positions are updated
for i in range(duration[0], duration[1]):
    # ...
    # change positions of obstacles
    # ...
    draw_params['time_begin'] = i
    redraw_obstacles(scenario, handles=handles, figure_handle=fig, plot_limits=plot_limits, draw_params=draw_params)
    print('fps:', (i - duration[0])/(time.time()-t1))

fps: 0.0


/home/rong/anaconda3/envs/flow/lib/python3.6/site-packages/matplotlib/figure.py:2362: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "
